# Credit Risk Resampling Techniques

In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [38]:
# Load the data
file_path = Path('Resources/lending_data.csv')
df = pd.read_csv(file_path)

In [39]:
#Dummy encode homeowner column
df_encoded = pd.get_dummies(df, columns=["homeowner"])
df_encoded

loan_size  interest_rate  borrower_income  debt_to_income  \
0        10700.0          7.672            52800        0.431818   
1         8400.0          6.692            43600        0.311927   
2         9000.0          6.963            46100        0.349241   
3        10700.0          7.664            52700        0.430740   
4        10800.0          7.698            53000        0.433962   
...          ...            ...              ...             ...   
77531    19100.0         11.261            86600        0.653580   
77532    17700.0         10.662            80900        0.629172   
77533    17600.0         10.595            80300        0.626401   
77534    16300.0         10.068            75300        0.601594   
77535    15600.0          9.742            72300        0.585062   

       num_of_accounts  derogatory_marks  total_debt loan_status  \
0                    5                 1       22800    low_risk   
1                    3                 0       13600    low_risk   
2                    3                 0       16100    low_risk   
3                    5                 1       22700    low_risk   
4                    5                 1       23000    low_risk   
...                ...               ...         ...         ...   
77531               12                 2       56600   high_risk   
77532               11                 2       50900   high_risk   
77533               11                 2       50300   high_risk   
77534               10                 2       45300   high_risk   
77535                9                 2       42300   high_risk   

       homeowner_mortgage  homeowner_own  homeowner_rent  
0                       0              1               0  
1                       0              1               0  
2                       0              0               1  
3                       0              1               0  
4                       1              0               0  
...                   ...            ...             ...  
77531                   0              1               0  
77532                   1              0               0  
77533                   0              0               1  
77534                   1              0               0  
77535                   1              0               0  

[77536 rows x 11 columns]

# Split the Data into Training and Testing

In [40]:
# Create our features
X = df_encoded.drop(columns="loan_status")

# Create our target
y = df_encoded["loan_status"]

In [41]:
X.describe()

loan_size  interest_rate  borrower_income  debt_to_income  \
count  77536.000000   77536.000000     77536.000000    77536.000000   
mean    9805.562577       7.292333     49221.949804        0.377318   
std     2093.223153       0.889495      8371.635077        0.081519   
min     5000.000000       5.250000     30000.000000        0.000000   
25%     8700.000000       6.825000     44800.000000        0.330357   
50%     9500.000000       7.172000     48100.000000        0.376299   
75%    10400.000000       7.528000     51400.000000        0.416342   
max    23800.000000      13.235000    105200.000000        0.714829   

       num_of_accounts  derogatory_marks    total_debt  homeowner_mortgage  \
count     77536.000000      77536.000000  77536.000000        77536.000000   
mean          3.826610          0.392308  19221.949804            0.497472   
std           1.904426          0.582086   8371.635077            0.499997   
min           0.000000          0.000000      0.000000            0.000000   
25%           3.000000          0.000000  14800.000000            0.000000   
50%           4.000000          0.000000  18100.000000            0.000000   
75%           4.000000          1.000000  21400.000000            1.000000   
max          16.000000          3.000000  75200.000000            1.000000   

       homeowner_own  homeowner_rent  
count   77536.000000    77536.000000  
mean        0.398911        0.103616  
std         0.489678        0.304764  
min         0.000000        0.000000  
25%         0.000000        0.000000  
50%         0.000000        0.000000  
75%         1.000000        0.000000  
max         1.000000        1.000000

In [42]:
# Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [43]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=70)
X_train

loan_size  interest_rate  borrower_income  debt_to_income  \
65978     8900.0          6.908            45600        0.342105   
43918    11300.0          7.933            55300        0.457505   
49003     7000.0          6.089            37900        0.208443   
69141     9800.0          7.296            49300        0.391481   
4538     10500.0          7.580            51900        0.421965   
...          ...            ...              ...             ...   
47733    10000.0          7.379            50000        0.400000   
70997     8300.0          6.647            43100        0.303944   
45492    10900.0          7.762            53600        0.440299   
21563     8600.0          6.778            44400        0.324324   
25916     9100.0          7.002            46500        0.354839   

       num_of_accounts  derogatory_marks  total_debt  homeowner_mortgage  \
65978                3                 0       15600                   1   
43918                5                 1       25300                   1   
49003                1                 0        7900                   0   
69141                4                 0       19300                   0   
4538                 4                 1       21900                   0   
...                ...               ...         ...                 ...   
47733                4                 1       20000                   1   
70997                2                 0       13100                   1   
45492                5                 1       23600                   1   
21563                3                 0       14400                   1   
25916                3                 0       16500                   1   

       homeowner_own  homeowner_rent  
65978              0               0  
43918              0               0  
49003              1               0  
69141              1               0  
4538               0               1  
...              ...             ...  
47733              0               0  
70997              0               0  
45492              0               0  
21563              0               0  
25916              0               0  

[58152 rows x 10 columns]

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [44]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [45]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [46]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Simple Logistic Regression

In [47]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [48]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9836709400786279

In [49]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,    17],
       [  108, 18644]])

In [50]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.97      0.99      0.91      0.98      0.97       632
   low_risk       1.00      0.99      0.97      1.00      0.98      0.97     18752

avg / total       0.99      0.99      0.97      0.99      0.98      0.97     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [51]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56284, 'high_risk': 56284})

In [52]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [53]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)

balanced_accuracy_score(y_test, y_pred) 

0.9937424396250054

In [54]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred) 

array([[  628,     4],
       [  116, 18636]])

In [55]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       632
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18752

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [56]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'low_risk': 56284, 'high_risk': 56284})

In [57]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [58]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9937424396250054

In [59]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  628,     4],
       [  116, 18636]])

In [60]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.84      0.99      0.99      0.91      0.99      0.99       632
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18752

avg / total       0.99      0.99      0.99      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [61]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_resampled) 

Counter({'high_risk': 1868, 'low_risk': 1868})

In [62]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [63]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test_scaled)

balanced_accuracy_score(y_test, y_pred) 

0.9907911932431849

In [64]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  624,     8],
       [  108, 18644]])

In [65]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.99      0.99      0.91      0.99      0.98       632
   low_risk       1.00      0.99      0.99      1.00      0.99      0.98     18752

avg / total       0.99      0.99      0.99      0.99      0.99      0.98     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [66]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# View the count of target classes with Counter
Counter(y_resampled)

Counter({'high_risk': 74105, 'low_risk': 74574})

In [67]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [68]:
# Calculate the balanced accuracy score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred) 

0.8046692740959951

In [69]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred) 

array([[  628,     4],
       [ 7207, 11545]])

In [70]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred)) 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.08      0.99      0.62      0.15      0.78      0.63       632
   low_risk       1.00      0.62      0.99      0.76      0.78      0.59     18752

avg / total       0.97      0.63      0.98      0.74      0.78      0.59     19384



# Final Questions

1. Which model had the best balanced accuracy score?

   SMOTE Oversampling.

2. Which model had the best recall score?

    Both Oversampling models and the Undersampling model.

3. Which model had the best geometric mean score?

    Both Oversampling models and the Undersampling model.
